In [ ]:
!pip install gradio
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install evaluate
!pip install google-colab

from IPython.display import clear_output
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import evaluate
import matplotlib.pyplot as plt
from transformers import TrainerCallback, TrainerControl, TrainerState
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import google.colab
import math
import torch
import pandas as pd
import gradio as gr

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sample"], padding="max_length", truncation=True)

def myPred(trainer, model, tokenizer, sample):
  df = pd.DataFrame({'sample': sample})
  df.to_csv(path+'testSample.csv', index=False)
  ds = load_dataset('csv', data_files=path+'testSample.csv')
  model.eval()
  tokenizer.pad_token = tokenizer.eos_token
  model.config.pad_token_id = model.config.eos_token_id

  tokenized_datasets = ds.map(tokenize_function, batched=False)

  testExample = tokenized_datasets["train"]
  testExample_preds = trainer.predict(testExample)
  testExample_preds = np.argmax(testExample_preds.predictions, axis=1)
  return testExample_preds

In [ ]:
drive = google.colab.drive.mount('/content/drive')
path = '/content/drive/MyDrive/ECE1786/'
model = AutoModelForSequenceClassification.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

trainer = Trainer(
  model=model
)

In [ ]:
sample = ["I had to take a ferry to reach Barrie as it's located on an island.  Collingwood is a town in Simcoe County, Ontario, Canada. Merritt has dozens of bronzed hand prints of country music stars who have been in Merritt for the annual Merritt Mountain Music Festival displayed throughout town."]

output = myPred(trainer, model, tokenizer, sample)
output[0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

0

In [ ]:
import gradio as gr
def getResult(input1, input2, input3):
  input = [input1+" "+input2+" "+input3]
  output = myPred(trainer, model, tokenizer, input)
  pred = str(output[0])

  result = ''
  if(pred == '0'):
    result = "The first sentence is a lie!"
  elif(pred == '1'):
    result = "The second sentence is a lie!"
  elif(pred == '2'):
    result = "The third sentence is a lie!"

  return result

with gr.Blocks() as demo:
  with gr.Column(scale=1, min_width=600):
    text0 = gr.Textbox(label="Sentence 1")
    text1 = gr.Textbox(label="Sentence 2")
    text2 = gr.Textbox(label="Sentence 3")
    predict = gr.Button("Submit")
    text3 = gr.Textbox(label="Prediction")

  predict.click(getResult, inputs=[text0, text1, text2], outputs=text3)

demo.launch(debug=True)